# TimeSeriesForecasting_Databricks

#### *Parts of this notebook have been obfuscated to protect privacy and security*

#### *All outputs have also been cleared*

In [ ]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list 
apt-get update
ACCEPT_EULA=Y apt-get install msodbcsql17
apt-get -y install unixodbc-dev
sudo apt-get install python3-pip -y
pip3 install --upgrade pyodbc

In [ ]:
import os

import pandas as pd
import pyodbc as dbc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#set up connection to SQL prod server
prod_conn = dbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                       'SERVER=SERVER_NAME;'
                       'DATABASE=DATABASE_NAME;UID=USERNAME;'
                       'PWD=PASSWORD')

In [ ]:
query = '''
select MIN(TRANSACTION_DATE) minimum_date, MAX(TRANSACTION_DATE) maximum_date, COUNT(*) number_of_rows
FROM DATABASE_NAME.TABLE_NAME
WHERE ORG_ID = 87
'''
date_range = pd.read_sql_query(query, prod_conn)
date_range

In [ ]:
query = '''
select TOP 7 *
FROM DATABASE_NAME.TABLE_NAME
WHERE ORG_ID = 87
'''
orders = pd.read_sql_query(query, prod_conn)
orders

In [ ]:
orders.columns

In [ ]:
# date
# store
# item
# $

other_items = ['BUSINESS_UNIT_NAME', 'PRODUCT_FAMILY_NAME', 'PRODUCT_CLASS_CODE', 'PRODUCT_CLASS_CODE_DESC', 'PRODUCT_GROUP_CODE_DESC', 'REPORT_CATEGORY_CODE_DESC', 'MAJOR_CODE', 'MAJOR_CODE_DESC']

other_stores = ['ALIGNSALESREPID', 'ALIGNTERRITORYID', 'ALIGNREGIONID']

date = 'TRANSACTION_DATE'
store = 'ORGID_SHIPTO_ORGID'
item = 'ITEM_NUMBER'
amount = 'TOTAL_ORDERS'

In [ ]:
query = '''
select TOP 7 {date}, {store}, {item}, SUM({amount}) total_amount
FROM DATABASE_NAME.TABLE_NAME
WHERE ORG_ID = 87
GROUP BY {date},
{store},
{item}
ORDER BY
{date}
;
'''
query = query.format(date=date, store=store, item=item, amount=amount)
orders = pd.read_sql_query(query, prod_conn)
orders
#print(query)

# Find the Top selling item (highest quantity)

In [ ]:
query = '''
select {item}, SUM(TOTAL_QTY) total_qty
FROM DATABASE_NAME.TABLE_NAME
WHERE ORG_ID = 87
GROUP BY
{item}
ORDER BY
total_qty DESC;
'''
query = query.format(date=date, store=store, item=item, amount=amount)
top_selling_item = pd.read_sql_query(query, prod_conn)
top_selling_item

In [ ]:
prod_conn = dbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                       'SERVER=SERVER_NAME;'
                       'DATABASE=DATABASE_NAME;UID=USERNAME;'
                       'PWD=PASSWORD')

In [ ]:
query = '''
select TOP 7 *
FROM DATABASE_NAME.TABLE_NAME
WHERE ORG_ID = 87;
'''

In [ ]:
query = '''
SELECT TRANSACTION_DATE, ORGID_SHIPTO_ORGID, ITEM_NUMBER, SUM(TOTAL_QTY) total_qty, SUM(TOTAL_ORDERS) total_amount
FROM DATABASE_NAME.TABLE_NAME
WHERE ORG_ID = 87
GROUP BY
TRANSACTION_DATE,
ORGID_SHIPTO_ORGID,
ITEM_NUMBER
'''

In [ ]:
import pyodbc
#import pyspark
#sc = pyspark.SparkContext()
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=SERVER_NAME;DATABASE=DATABASE_NAME;UID=USERNAME;PWD=PASSWORD')
cur = conn.cursor()
cur.execute(query)
results = cur.fetchall()

#df = sc.parallelize(results).toDF()

#df.saveAsTable('my_spark_table')
cur.close()
conn.close()
results[0:6]

In [ ]:
len(results)

In [ ]:
jdbc_url = "jdbc:sqlserver://SERVER_NAME;database=DATABASE_NAME;user=USERNAME;password=PASSWORD"

train = spark.read.format("jdbc").option("url", jdbc_url).option("query", query).load()
train

In [ ]:
# make the dataframe queryable as a temporary view
train.createOrReplaceTempView('train')

display(train)

In [ ]:
%sql
 
SELECT
  ORGID_SHIPTO_ORGID as customer,
  ITEM_NUMBER as item, 
  sum(total_qty) as quantity
FROM train
GROUP BY customer, item
ORDER BY quantity DESC;

# Most popular Customer, Item combination is CUSTOMER_NO and XXX123456

In [ ]:
%sql
 
SELECT
  year(TRANSACTION_DATE) as year, 
  sum(total_qty) as sales
FROM train
GROUP BY year(TRANSACTION_DATE)
ORDER BY year;

In [ ]:
import matplotlib.pyplot as plt

# Convert the Spark DataFrame to pandas DataFrame
pandas_df = _sqldf.toPandas()

# Plot a line chart of the sales by year
plt.plot(pandas_df['year'].to_numpy(), pandas_df['sales'].to_numpy())

# Set the chart title and axis labels
plt.title('Sales by Year')
plt.xlabel('Year')
plt.ylabel('Sales')

# Display the plot
plt.show()

In [ ]:
pandas_df

# Build a single forecast first
## Retrieve Data for a Single Item-Customer Combination

In [ ]:
%sql

SELECT
    CAST(TRANSACTION_DATE as date) as ds,
    total_qty as y
  FROM train
  WHERE ORGID_SHIPTO_ORGID='876086' AND ITEM_NUMBER='CAT01586'
  ORDER BY ds

In [ ]:
# query to aggregate data to date (ds) level
sql_statement = '''
  SELECT
    CAST(TRANSACTION_DATE as date) as ds,
    total_qty as y
  FROM train
  WHERE ORGID_SHIPTO_ORGID='CUSTOMER_NO'
  AND ITEM_NUMBER='ITEM_NO'
  AND TRANSACTION_DATE < '2023-01-01'
  ORDER BY ds
  '''
 
# assemble dataset in Pandas dataframe
history_pd = spark.sql(sql_statement).toPandas()
 
# drop any missing records
history_pd = history_pd.dropna()

In [ ]:
history_pd

In [ ]:
from prophet import Prophet
import logging
 
# disable informational messages from prophet
logging.getLogger('py4j').setLevel(logging.ERROR)

# Train the Prohpet model

In [ ]:
# set model parameters
model = Prophet(
  interval_width=0.95,
  growth='linear',
  daily_seasonality=False,
  weekly_seasonality=True,
  yearly_seasonality=True,
  seasonality_mode='multiplicative'
  )
 
# fit the model to historical data
model.fit(history_pd)

# Build Forecast

In [ ]:
# define a dataset including both historical dates & 90-days beyond the last available date
future_pd = model.make_future_dataframe(
  periods=90, 
  freq='d', 
  include_history=True
  )
 
# predict over the dataset
forecast_pd = model.predict(future_pd)
 
#display(forecast_pd)
forecast_pd

In [ ]:
type(forecast_pd)

In [ ]:

import matplotlib.pyplot as plt

# Get the components of the forecast
trend = forecast_pd['trend']
yearly_seasonality = forecast_pd['yearly']
weekly_seasonality = forecast_pd['weekly']

# Plot the components
fig, axs = plt.subplots(3, 1, figsize=(12, 6))

axs[0].plot(trend)
axs[0].set_title('Trend')

axs[1].plot(yearly_seasonality)
axs[1].set_title('Yearly Seasonality')

axs[2].plot(weekly_seasonality)
axs[2].set_title('Weekly Seasonality')

fig.tight_layout()

# Display the plot
plt.show()

In [ ]:
forecast_pd.iloc[75:95]

In [ ]:
predict_fig = model.plot( forecast_pd, xlabel='date', ylabel='sales')
 
# adjust figure to display dates from last year + the 90 day forecast
xlim = predict_fig.axes[0].get_xlim()
new_xlim = ( xlim[1]-(180.0+365.0), xlim[1]-90.0)
predict_fig.axes[0].set_xlim(new_xlim)
 
display(predict_fig)